In [16]:
%reload_ext dotenv
%dotenv

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [18]:
import os

In [19]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [20]:
chat = ChatGoogleGenerativeAI(
    google_api_key = os.getenv("GEMINI_API_KEY"),  
    model = "gemini-2.0-flash",
    temperature = 0,
    max_output_tokens = 500,
    model_kwargs = {"seed": 365}
)

In [21]:
string_parser = StrOutputParser()

In [22]:
chain_books = chat_template_books | chat | string_parser
chain_projects = chat_template_projects | chat | string_parser

In [23]:
chain_parallel = RunnableParallel({'books': chain_books,
'projects':chain_projects})

In [25]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '*   Fluent Python\n*   Effective Python: 90 Specific Ways to Write Better Python\n*   Python Cookbook',
 'projects': '*   **Web Scraper with Data Analysis and Visualization**\n*   **Personal Finance Tracker with GUI**\n*   **Simple Game Engine with Custom Game**'}

In [26]:
chain_time1 = (RunnableParallel({'books':chain_books,
                                'projects':chain_projects})
              | chat_template_time
              | chat
              | string_parser)

In [27]:
chain_time2 = ({'books': chain_books,
               'projects':chain_projects}
              | chat_template_time
              | chat
              | string_parser
              )

In [28]:
print(chain_time2.invoke({'programming language':'Python'}))

Okay, let's break down the estimated time commitment for each book and project, considering you're an intermediate programmer.  Keep in mind these are *very rough* estimates, and your actual time will vary based on your learning style, focus, and prior experience with specific libraries or concepts.

**Literature Time Estimates:**

*   **Fluent Python (Luciano Ramalho):** This is a deep dive into Python's core features and idioms. It's not a quick read.  It's best approached by reading a section, experimenting with the code, and then moving on.

    *   **Estimated Time:** 80-120 hours.  This assumes you're actively coding along with the examples and doing the exercises.  You might spend 2-4 hours per chapter, depending on the complexity.  It's a book you'll likely revisit over time.

*   **Effective Python: 90 Specific Ways to Write Better Python (Brett Slatkin):** This book is more digestible. Each "way" is a self-contained tip. You can read it in chunks.

    *   **Estimated Time:**

In [29]:
chain_time2.get_graph().print_ascii()

                   +-------------------------------+                     
                   | Parallel<books,projects>Input |                     
                   +-------------------------------+                     
                       ****                  ****                        
                   ****                          ****                    
                 **                                  **                  
  +--------------------+                       +--------------------+    
  | ChatPromptTemplate |                       | ChatPromptTemplate |    
  +--------------------+                       +--------------------+    
             *                                            *              
             *                                            *              
             *                                            *              
+------------------------+                   +------------------------+  
| ChatGoogleGenerativeAI |            